# Crawl listing information

In [ ]:
#Import appropriate libraries
import pandas as pd
import urllib.request #urllib request is a url communication library
import time #library to add delay to counter the anti-crawling algorithm
from bs4 import BeautifulSoup # beautiful soup library
#Data parse of of all the district in shanghai

df_neighborhood = pd.read_csv("C:/Users/melvi/Desktop/Capstone/Section_Raw.csv", encoding = "gbk")

section_list = df_neighborhood["Section"]
neighborhood1_list = section_list[1:21]


section_in_chinese = df_neighborhood["Section in Chinese"]

def page_counter(url):
    
    page =  BeautifulSoup(url, 'lxml')
    listing = BeautifulSoup(url, 'lxml')
    listing_number = listing.select("#content > div.leftContent > div.resultDes.clear > h2 > span")
    listing_number = int(str(listing_number[0]).split(">")[1].split("<")[0])
    if listing_number == 0:
        page_count = 0
    else: 
        page_count = page.select('#content > div.leftContent > div.contentBottom.clear > div.page-box.fr > div')
        page_count = str(page_count[0]).split(":")[1].split(",")[0]
        
    return page_count

def parse_HTMLdata(htmlstr):
    #create a beautifulsoup object, the second argument is the engine.
    sp =  BeautifulSoup(htmlstr, 'lxml')
    
    #get the housing information
    
    house_list = sp.select('#content > div.leftContent > ul > li')
    page_list = []
    
    district = BeautifulSoup(htmlstr, 'lxml')
    district = district.select('body > div:nth-of-type(3) > div > div.position > dl:nth-of-type(2)  > dd > div:nth-of-type(1) > div:nth-of-type(1) > a.selected')
    district = str(district[0]).split(">")[1].split("<")[0]
    

    
    for house in house_list:
    
        rows_list = []
        
        title = house.select('div.info.clear > div.title > a')
        title = (title[0].text).strip()
        rows_list.append(title)

        
        infos = house.select('div.info.clear > div.address > div')
        infos = (infos[0].text).strip()
        layout = infos.split("|")[0]
        area = infos.split("|")[1]
        orientation = infos.split("|")[2]
        decoration = infos.split("|")[3]
        floor = infos.split("|")[4]
        year = infos.split("|")[5]
        house_type = infos.split("|")[6]

        rows_list.append(layout)  
        rows_list.append(area)
        rows_list.append(orientation)
        rows_list.append(decoration)
        rows_list.append(floor)
        rows_list.append(year)
        rows_list.append(house_type)
 

        addr_community = house.select('div.info.clear > div.flood > div > a:nth-of-type(1)')
        addr_community = (addr_community[0].text).strip()
        rows_list.append(addr_community)   
 
        addr_neighborhood = house.select('div.info.clear > div.flood > div > a:nth-of-type(2)')
        addr_neighborhood = (addr_neighborhood[0].text).strip()
        rows_list.append(addr_neighborhood)   

        url_community = house.select('div.info.clear > div.flood > div > a:nth-of-type(1)')
        url_community = str(url_community[0]).split("/")[4]
        rows_list.append(url_community)    


        rows_list.append(district)
        
        total_price = house.select('div.info.clear > div.priceInfo > div.totalPrice > span')
        total_price = (total_price[0].text).strip()
        rows_list.append(total_price)   

        unit_price = house.select('div.info.clear > div.priceInfo > div.unitPrice > span')
        unit_price = (unit_price[0].text).strip()
        rows_list.append(unit_price)

        
        following_info = house.select('div.info.clear > div.followInfo')
        following_info = (following_info[0].text).strip()
        rows_list.append(following_info)
      
        
        subway_tag = house.select('div.info.clear > div.tag > span.subway')
        if subway_tag == []:
            subway_tag = 0
        else:
            subway_tag = 1
        rows_list.append(subway_tag)
   
        good_tag = house.select('div.info.clear > div.tag > span.good')
        if good_tag == []:
            good_tag = 0
        else:
            good_tag = 1
        rows_list.append(good_tag)
    
        vr_tag = house.select('div.info.clear > div.tag > span.vr')
        if vr_tag == []:
            vr_tag = 0
        else:
            vr_tag = 1
        rows_list.append(vr_tag)
        
        taxfree_tag = house.select('div.info.clear > div.tag > span.taxfree')
        if taxfree_tag == []:
            taxfree_tag = 0
        else:
            taxfree_tag = 1
        rows_list.append(taxfree_tag)
        
        haskey_tag = house.select('div.info.clear > div.tag > span.haskey')
        if haskey_tag == []:
            haskey_tag = 0
        else:
            haskey_tag = 1
        rows_list.append(haskey_tag)
        
        five_tag = house.select('div.info.clear > div.tag > span.five')
        if five_tag == []:
            five_tag = 0
        else:
            five_tag = 1
        rows_list.append(five_tag)

        
        page_list.append(rows_list)
    
    return page_list


#Data crawling of of all the district in shanghai
def request_data(url):
    
    '''crawl current website data,parameter is the website address, return data list'''
   
    #create request object
    req = urllib.request.Request(url)
    
    #use with as expression to save resouces.
    page_data_list = []
    
    with urllib.request.urlopen(req) as response:
        #get the string object
        data = response.read()
        htmlstr = data.decode()
        L = parse_HTMLdata(htmlstr)
        #use extend instead of append, as we don't want nesting
        page_data_list.extend(L)
    
    return page_data_list

def request_count(url):
    req = urllib.request.Request(url)
    
    with urllib.request.urlopen(req) as response:
        #get the string object
        data = response.read()
        htmlstr = data.decode()
        count = page_counter(htmlstr)
        #use extend instead of append, as we don't want nesting
 
    return count


    

#Lianjia Shanghai second-hand housing website
url_temp = 'https://sh.lianjia.com/ershoufang/{}/pg{}/'

#create a data list
data_list = []
neighborhood1_list = ["beicai"]
for neighborhood in neighborhood1_list:
    count = request_count(url_temp.format(neighborhood,1))
    print(count)
    i = 1
    while i <= int(count):
        url = url_temp.format(neighborhood,i)
        print(url)
        print("++++++++++++{} neighborhood:Page{}++++++++++++++".format(neighborhood,i))
        
        try :        
            # Counter anti-crawling
            # sleep 15 seconds
            time.sleep(15)
            L = request_data(url)
            data_list.extend(L)
        except Exception as e:
            print(e)
            time.sleep(20)
            try : 
                L = request_data(url)
                data_list.extend(L)
                continue
            except Exception as e1:
                print(e1)
                
            # stop looping
            print('Stop Looping')
            break
        i = i + 1
print('data_list2 =', len(data_list))
print('Finish')

#Save data
colsname = ['Title', 'layout','area','orientation','decoration','floor','year','house_type', 'Community', 'Neighborhood','Community URL','District', 'Total Price', 'Unit Price', 'Follow Infos','Subway','Good','VR','Five','Has Key','Two']   

df = pd.DataFrame(data_list, columns=colsname)    
df.to_csv('C:/Users/melvi/Desktop/Capstone/house_data4.csv', index=False, encoding='gbk')

print('Data Saved')

In [2]:
# Community info crawler

In [ ]:
import pandas as pd
import numpy as np
import urllib.request #urllib request is a url communication library
import time #library to add delay to counter the anti-crawling algorithm
from bs4 import BeautifulSoup # beautiful soup library
#Data parse of of all the district in shanghai

input_df = pd.read_excel('C:/Users/melvi/Desktop/Capstone/Missing Community.xlsx', encoding='gbk')

community_list = (input_df["ID"].unique())

community_list = pd.Series(community_list)

community_list = community_list.drop_duplicates()

community_list_int = []

for num in community_list:
    community_list_int.append(int(num))
    
    

#community_list.to_csv('C:/Users/melvi/Desktop/Capstone/community_list.csv', index=False, encoding='gbk')

# -*- coding: utf-8 -*-
"""
Created on Sat Nov  9 19:05:22 2019

@author: melvi
"""
def parse_HTMLdata(htmlstr):
    #create a beautifulsoup object, the second argument is the engine.
    sp =  BeautifulSoup(htmlstr, 'lxml')
    
    #get the housing information
    
    house_list = sp.select('body')
    page_list = []


    
    for house in house_list:
    
        rows_list = []
        
        Community_Name = house.select('div.xiaoquDetailHeader > div > div.detailHeader.fl > h1')
        Community_Name = str(Community_Name[0]).split(">")[1].split("<")[0]
        rows_list.append(Community_Name)
        
        Community_Address = house.select('div.xiaoquDetailHeader > div > div.detailHeader.fl > div')
        Community_Address = str(Community_Address[0]).split(">")[1].split("<")[0]
        rows_list.append(Community_Address)
        
        rows_list.append(Community)
        
        Community_Price = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquPrice.clear > div > span.xiaoquUnitPrice')
        Community_Price = (Community_Price[0].text).strip()
        rows_list.append(Community_Price)
        
        
        Build_Year = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(1) > span.xiaoquInfoContent')
        Build_Year = (Build_Year[0].text).strip()
        rows_list.append(Build_Year)
        
        Buiding_Type = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(2) > span.xiaoquInfoContent')
        Buiding_Type = (Buiding_Type[0].text).strip()
        rows_list.append(Buiding_Type)
        
        Maintenance_Cost= house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(3) > span.xiaoquInfoContent')
        Maintenance_Cost = (Maintenance_Cost[0].text).strip()
        rows_list.append(Maintenance_Cost)
        
        Company = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(4) > span.xiaoquInfoContent')
        Company = (Company[0].text).strip()
        rows_list.append(Company)
        
        Developer = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(5) > span.xiaoquInfoContent')
        Developer = (Developer[0].text).strip()
        rows_list.append(Developer)
        
        Total_Building = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(6) > span.xiaoquInfoContent')
        Total_Building = (Total_Building[0].text).strip()
        rows_list.append(Total_Building)
        
        Total_House = house.select('div.xiaoquOverview > div.xiaoquDescribe.fr > div.xiaoquInfo > div:nth-of-type(7) > span.xiaoquInfoContent')
        Total_House = (Total_House[0].text).strip()
        rows_list.append(Total_House)

        page_list.append(rows_list)
    
    return page_list


#Data crawling of of all the district in shanghai
def request_data(url):
    
    '''crawl current website data,parameter is the website address, return data list'''
   
    #create request object
    req = urllib.request.Request(url)
    
    #use with as expression to save resouces.
    page_data_list = []
    
    with urllib.request.urlopen(req) as response:
        #get the string object
        data = response.read()
        htmlstr = data.decode()
        L = parse_HTMLdata(htmlstr)
        #use extend instead of append, as we don't want nesting
        page_data_list.extend(L)
    
    return page_data_list



#Lianjia Shanghai second-hand housing website
url_temp = 'https://sh.lianjia.com/xiaoqu/{}/'

#create a data list
data_list = []
i = 0
sleep_time = 2
for Community in community_list:
    url = url_temp.format(Community)
    print(url)
    print("++++++++++++{} Community++++++++++++++".format(Community))
        
    try:        
        # Counter anti-crawling
        # sleep 10 seconds
        time.sleep(sleep_time)
        L = request_data(url)
        data_list.extend(L)
    except Exception as e:
        print(e)
        time.sleep(sleep_time)
        Community = Community + 1
        try: 
            L = request_data('https://sh.lianjia.com/xiaoqu/{}/'.format(Community))
            data_list.extend(L)
        except Exception as e10:
            print(e)
            time.sleep(sleep_time)
            Community = Community + 1
            try: 
                L = []
                data_list.extend(L)
                continue
            except Exception as e10:
                print(e10)
                print('Stop Looping')
                break
    print(i)
    i = i + 1

print('data_list2 =', len(data_list))
print('Finish')
#Save data
colsname = ['Community Name','Community Address','Community','Community Price','Build Year','Buiding_Type','Maintenance_Cost','Company','Developer','Total_Building','Total_House']   
        

df = pd.DataFrame(data_list, columns=colsname)    
df.to_csv('C:/Users/melvi/Desktop/Capstone/Filled Community.csv', index=False, encoding='gbk')

print('Data Saved')

# Connect to BaiduMap API to obtain lattitude and longitude of each community

In [ ]:


import pandas as pd
import json
from urllib.request import urlopen,quote

geocoder = pd.DataFrame(columns=['Community','Latitude','Longitude'])
latitude = []
longitude = []
community_list = []
def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoding/v3/?address='
    output = 'json'
    ak = 'myak'
    add = quote(address)
    url2 = url+add+'&output='+output+"&ak=" +ak
    req = urlopen(url2)
    res  = req.read().decode()
    temp = json.loads(res)
    lng = temp['result']['location']['lng']
    lat = temp['result']['location']['lat']

    community_list.append(community)
    latitude.append(lat)
    longitude.append(lng)

data_1 = pd.read_excel("C:/Users/melvi/Desktop/Capstone/Community info.xlsx", encoding = "gbk")
i = 0
for community in data_1["Community"]:
    community=community.strip()
    getlnglat(community)
    i = i + 1
    print(i)
    
geocoder["Community"] = community_list
geocoder["Latitude"] = latitude
geocoder["Longitude"] = longitude
    

geocoder.to_csv("C:/Users/melvi/Desktop/Capstone/geocode.csv",encoding = "gbk")    


# get facility information within 1km raidius

In [ ]:
import pandas as pd
import json
from urllib.request import urlopen,quote
import math
import time

place = pd.DataFrame(columns=['Community','Page','Name','Distance','Tag'])
name_list = []
community_list = []
distance_list = []
tag_list = []
page_list = []

data_1 = pd.read_csv("C:/Users/melvi/Desktop/Capstone/geocode.csv", encoding = "gbk")
n=1020
m=2
place_list = data_1[n:]
i = n
for community in place_list['Community']:
    print(community)
    geocode = "{},{}".format(place_list["Latitude"][i],place_list["Longitude"][i])
    time.sleep(0)
    count=0
    url = 'http://api.map.baidu.com/place/v2/search?query='
    location= geocode
    radius = str(1000)
    output = 'json'
    ak = 'myak'
    query = quote("购物中心$公园$中学$小学$综合医院$专科医院$地铁站")
    #'&tag=' +tag+
    url2 = url+query+ '&location='+ location + "&radius="+radius+ "&output="+output+"&scope=2"+'&page_size=20'+"&ak="+ak
    req = urlopen(url2)
    res  = req.read().decode()
    temp = json.loads(res)
    print(temp)
    page_num = math.ceil(int(temp['total'])/20) - 1
    page_count = 0
    try:
        count_limit = len(temp['results'])
        count=0
        while count < count_limit:
            name = temp['results'][count]['name']
            distance = temp['results'][count]['detail_info']['distance']
            tag = temp['results'][count]['detail_info']['tag']
            count = count + 1
            name_list.append(name)
            distance_list.append(distance)
            tag_list.append(tag)
            community_list.append(community)
            page_list.append(page_count)

        
        page_count = 1
        while page_count <= page_num:
            page_count_str = str(page_count)
            count=0
            url3 = url+query+ '&location='+ location + "&radius="+radius+ "&output="+output+"&scope=2"+'&page_size=20'+'&page_num='+page_count_str+"&ak="+ak
            req = urlopen(url3)
            res  = req.read().decode()
            temp = json.loads(res)

            count_limit = len(temp['results'])
            count=0
            while count < count_limit:
                name = temp['results'][count]['name']
                distance = temp['results'][count]['detail_info']['distance']
                tag = temp['results'][count]['detail_info']['tag']
                count = count + 1
                name_list.append(name)
                distance_list.append(distance)
                tag_list.append(tag)
                community_list.append(community)
                page_list.append(page_count)
            page_count = page_count + 1
    except Exception as e:
        print(e)
        try : 
            count_limit = len(temp['results'])
            count=0
            while count < count_limit:
                print(count)
                name = temp['results'][count]['name']
                distance = temp['results'][count]['detail_info']['distance']
                tag = temp['results'][count]['detail_info']['tag']
                count = count + 1
                name_list.append(name)
                distance_list.append(distance)
                tag_list.append(tag)
                community_list.append(community)
                page_list.append(page_count)
    
            
            page_count = 1
            while page_count <= page_num:
                page_count_str = str(page_count)
                url3 = url+query+ '&location='+ location + "&radius="+radius+ "&output="+output+"&scope=2"+'&page_size=20'+'&page_num='+page_count_str+"&ak="+ak
                req = urlopen(url3)
                res  = req.read().decode()
                temp = json.loads(res)
                count_limit = len(temp['results'])
                count=0
                while count < count_limit:
                    name = temp['results'][count]['name']
                    distance = temp['results'][count]['detail_info']['distance']
                    tag = temp['results'][count]['detail_info']['tag']
                    count = count + 1
                    name_list.append(name)
                    distance_list.append(distance)
                    tag_list.append(tag)
                    community_list.append(community)
                    page_list.append(page_count)
                page_count = page_count + 1
                
        except Exception as e1:
            print(e1)
            try : 
                count_limit = len(temp['results'])
                count=0
                while count < count_limit:
                    print(count)
                    name = temp['results'][count]['name']
                    distance = temp['results'][count]['detail_info']['distance']
                    tag = []
                    count = count + 1
                    name_list.append(name)
                    distance_list.append(distance)
                    tag_list.append(tag)
                    community_list.append(community)
                    page_list.append(page_count)
    
            
                page_count = 1
                while page_count <= page_num:
                    page_count_str = str(page_count)
                    url3 = url+query+ '&location='+ location + "&radius="+radius+ "&output="+output+"&scope=2"+'&page_size=20'+'&page_num='+page_count_str+"&ak="+ak
                    req = urlopen(url3)
                    res  = req.read().decode()
                    temp = json.loads(res)
                    count_limit = len(temp['results'])
                    count=0
                    while count < count_limit:
                        name = temp['results'][count]['name']
                        distance = temp['results'][count]['detail_info']['distance']
                        tag = []
                        count = count + 1
                        name_list.append(name)
                        distance_list.append(distance)
                        tag_list.append(tag)
                        community_list.append(community)
                        page_list.append(page_count)
                    page_count = page_count + 1                
                    continue
                    
            except Exception as e2:
                print(e2)
          
                print('Stop Looping') 
                break    
        

    print(i)
    i = i + 1


place["Name"] = name_list
place["Community"] = community_list
place["Distance"] = distance_list
place["Tag"] = tag_list
place["Page"] = page_list

place.to_csv("C:/Users/melvi/Desktop/Capstone/place.csv".format(n),encoding = "gbk")    


# get the distance from the centre and nearest top shopping malls.

In [ ]:

import pandas as pd
import numpy as np
import math

df1 = pd.read_csv("C:/Users/melvi/Desktop/Capstone/final_1.csv",encoding = "gbk")


#calculate distance to the centre of Shanghai
EARTH_REDIUS = 6378.137

df_geo = pd.read_excel("C:/Users/melvi/Desktop/Capstone/geocoder.xlsx",encoding = "gbk")

def rad(d):
    return d * np.pi/ 180.0

def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(np.sin(a/2), 2) + np.cos(radLat1) * np.cos(radLat2) * math.pow(np.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s

centre =[31.236968,121.482341]
wu= [31.305458,121.522215]
xu = [31.201202,121.442314]
dong = [31.243738,121.490857]
xi = [31.236045,121.466545]
xin = [31.221998,121.481686]
lu = [31.243812,121.508984]
huan = [31.239028,121.419181]
zhong = [31.224734,121.425104]

i = 0
df_dist_centre = []
community_list = []
df_dist_shop = []

while i <8677:
    community = df_geo["Community"][i]
    community_list.append(community)
    lat1 = df_geo["Latitude"][i]
    lng1 = df_geo["Longitude"][i]
    lat2 = centre[0]
    lng2 = centre[1]
    dist_centre = getDistance(lat1, lng1, lat2, lng2)
    lat3 = wu[0]
    lng3 = wu[1]    
    lat4 = xu[0]
    lng4 = xu[1] 
    lat5 = dong[0]
    lng5 = dong[1] 
    lat6 = xin[0]
    lng6 = xin[1] 
    lat7 = lu[0]
    lng7 = lu[1] 
    lat8 = huan[0]
    lng8 = huan[1] 
    lat9 = zhong[0]
    lng9 = zhong[1]
    dist_1 = getDistance(lat1, lng1, lat3, lng3)
    dist_2 = getDistance(lat1, lng1, lat4, lng4)
    dist_3 = getDistance(lat1, lng1, lat5, lng5)
    dist_4 = getDistance(lat1, lng1, lat6, lng6)
    dist_5 = getDistance(lat1, lng1, lat7, lng7)
    dist_6 = getDistance(lat1, lng1, lat8, lng8)
    dist_7 = getDistance(lat1, lng1, lat9, lng9)
    dist_shop = min(dist_1,dist_2,dist_3,dist_4,dist_5,dist_6,dist_7)
    df_dist_shop.append(dist_shop)

    df_dist_centre.append(dist_centre)
    i = i+1
    print(i)
   
Distance = pd.DataFrame(columns=['Community','Distance centre',"Distance shop"]) 

Distance["Community"] = community_list
Distance["Distance centre"] = df_dist_centre
Distance["Distance shop"] = df_dist_shop

  

Distance.to_csv("C:/Users/melvi/Desktop/Capstone/distance.csv",index = False, encoding = "gbk")